# ACIT 2515 Midterm 1
### Topics Covered: 5. Pytest, 4. Decorators/Wrappers/Closures, 3. UV/venv, 2. Recursion 

#### Week 5 - Pytest

- First, must initialize the venv we're working inside - used with `uv init` or `uv init --bare` to avoid adding a `.git` file
- Pytest can be installed inside your virtual environment using `uv add --dev pytest`, installed as a dev dependency so it's not distributed 
    - Stage 1: Test Discovery
        - Pytest will only run these types of files = `test_*` or `*_test`
            - Inside these, will only run functions that match `test_*` naming convention or inside a `Test` class
    - Stage 2: Execution
        - After collecting, runs each function/method, executed in order they were discovered

- Pytest is used for automated testing, for error diagnosis and examination, and helps you catch bugs as you write code -- prevents bugs from reaching end users, and ensures changes don't break functionality
    - Therefore, test functions should be constructed before the functions themselves should be written
        - Tests should serve as documentation in this stage, to ensure the original function purpose is preserved - keeps us on track
        - Enforce discipline, to ensure we write code that meets requirements
        - We want different types of tests
            1. Red: write a failing test 
            2. Green: minimal code to make the test pass
            3. Test edge-cases, base case, thoroughly! 
            

- In particular, we are writing unit tests - they should be fast, isolated, repeatable, self-checking, automated tests 
    1. Arrange      ==>     set up data and condition
    2. Act          ==>     execute code being tested
    3. `assert`     ==>     verify the result is what we expect
    4. Cleanup      ==>     once tests are clear, delete created artifacts/states

- Test files typically import the functions from the self-created modules

In [ ]:
# Example code: simple add(a,b), would be in its own module (calculator.py)

def add(a,b):
    return a + b

# tests, would be in its own test_add.py
# test the edge, base case, and normal case

# from calculator import add

# normal case
def test_add():
    a = 10
    b = 20
    result = add(a,b)
    assert result == 30

# edge case
def test_add():
    a = 0
    b = 0
    result = add(a,b)
    assert result == 0

# edge case
def test_add():
    a = 0
    b = -2
    result = add(a,b)
    assert result == -2

# edge case
def test_add():
    a = -5
    b = -2
    result = add(a,b)
    assert result == -7

##### Asserts

- `assert` is a python keyword to test that a condition is true, raises `AssertionError` if condition is false
    - basically checks for a True statement

- Testing for exceptions

```python
def add_values(a, b):
    if type(a) is not int or type(b) is not int:
        raise TypeError("Invalid value")
    return a+b
```
We can catch expected exceptions using `with pytest.raises(<NAME>)` where `<NAME>`= Exception (in this case, `TypeError`)

```python
import pytest

def test_add_values_invalid():
  with pytest.raises(TypeError):
    result = add_values([1], [2])
```

##### Code Coverage

- Measures what % of code is executed by tests - industry standard of 80-90% coverage 
- Some pytest flags related to coverage  
        - `uv add --dev pytest-cov`   
        - `--cov=.` include in terminal output  
        - `--cov-report=html` makes a htmlcov/folder   
        - `--cov=calculator` for a module [calculator]  
        - `--cov=mypackage` for a given package  
        - `--cov=src` for a specific dir

##### Fixtures

- Reusable setup code for tests - can setup class instances, functions, **data**, so we don't have to rewrite everything every time we want to use it
        - example: I want to use the same string for all tests, use a fixture here instead
    - declared with `@pytest.fixture`

```python
@pytest.fixture
def username():
    return "alice"

@pytest.fixture
def email():
    return "alice@example.com"

@pytest.fixture
def age():
    return 30

def test_user_profile(username, email, age):
    profile = create_profile(username, email, age)
    assert profile.username == "alice"
    assert profile.email == "alice@example.com"
    assert profile.age == 30
```
- Decorate a function with `@pytest.fixture`, then function becomes the fixture name
- Then, pass this function as an argument into a test function
- Pytest automatically calls the fixture and passes the result to our test 
    - use fixtures to eliminate duplicate setup code
    - give fixtures descriptive names
    - use `tmp_path` for file operations in tests
    - put shared fixtures in `conftest.py` 

##### Builtin Pytest Fixtures
* `tmp_path`  
    - this creates a temporary subdir for each test, unique to each test function
* `capsys`  
    - captures what the code prints 

#### Week 4 - Decorators/Wrappers/Closures

##### args, kwargs
- `*args`
    - collects positional arguments into a tuple()

- `**kwargs`
    - collects keyward arguments into a dictionary
    - key:value pairs!
    - ** unpacks a dictionary into their keyword args for a function 

- order that these can be used in a function
    - `(normal_arg, *args, **kwargs)`


- in both of these, the * operator unpacks iterables into individual elements, so that you pass a list into a multi-parameter function

- Higher order functions 
    - functions that take a function as an argument, OR returns a function, OR **BOTH**
- Closure 
    - a nested function that can remember variables from it's enclosing scope even after being executed, stored in RAM
- Decorators
    - a callable function that takes a function, and returns a new function that adds behaviour before, after, or around the original function 
    - point is to modify or enhance a function without affecting the source code 
    - returning a function that suitable to replace the other
        - it returns a wrapper - the wrapper itself calls the original function to give us the result, does something with this result, returns what the original function should return
    - Decorator factory
        - allows us to further configure a decorator with more arguments

        ```python
        def repeat(times):
            """Decorator factory that repeats function output times times"""
            def decorator(func):
                def wrapper(*args, **kwargs):
                    result = func(*args, **kwargs)
                    return result * times
                return wrapper
            return decorator
        ```

    Basic structure
    ```python
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = func(args, kwargs)
            print(f"Logging result of {func}...")
            return result
        return wrapper
    ```
- Python variable scope
    1. L<sub>ocal</sub>
        - variable created inside a function, only callable in this function
    2. E<sub>nclosing scope</sub>
        -  scope for nested functions, containing names for both enclosing function and inner function (closures allow access to enclosing scope)
    3. G<sub>lobal scope</sub>
        - global to the module we are working in
    4. B<sub>uiltin</sub>
        - builtin variables, outermost scope and loaded when Python interpretor starts up (examples: )

- Example with nonlocal scope
    ```python
    def create_counter():
        count=0
        def increment():
            nonlocal count # Access enclosing scope variable 
            count += 1
            return count
        return increment
    ```
    - Why do we do this? because closures can remember variables from its enclosing scope even after the outside function is done executing -- by capturing thee variables, they can reference variables that persist even after the function ends
        - Closures remember things - they are stored in a special attribute called `__closure__` and live in memory
        - Decorators rely on closures to maintain state and access original functions being decorated

##### Applying Decorators to Funcs

- Basic structure
    ```python
    @decorator_logger
    def add_nums(a,b):
        return a + b
    ```

##### Function Metadata
- `function.__name__` will return `function`
- `function.__doc__` will return docstrings 
- `function.__module__` will return `__main__` if declared in the same working module

- `from functools import wraps`
- `@wraps(func)`
    - ensures we preserve the original function's metadata
    - A function will only take the highest order metadata, can lose og metadata since there are enclosed functions 


